In [ ]:
#Import Modules

import sys
sys.path.append("..") #give this script access to all modules in parent directory
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.io import loadmat
import scipy
from scipy.stats import skew
from Data_Cleaning import preprocess
from Data_Cleaning import preprocess_sickbay
import Filtering
import Actigraph_Metrics
from scipy.stats import mannwhitneyu


In [ ]:
data_dir = 'C:/Users/sidha/OneDrive/Sid Stuff/PROJECTS/iMEDS Design Team/Data Analysis/PedAccel/data_analysis/PythonPipeline/PatientData'
# data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData'
   # data_dir = r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\PatientData'
window_size = 100 #100 is 1 second worth of time
lead_time = 10
slice_size_min = 15
sr = .5

In [ ]:
        
#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
       # data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')           
        data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')
        
       # accel_data = loadmat(data_filepath_accel)
       # x_mag = accel_data["x_mag"]
       # accel_SBS = accel_data["sbs"].flatten()
        vitals_data = loadmat(data_filepath_vitals)
        temp_hr = vitals_data['heart_rate']
        temp_SpO2 = vitals_data['SpO2']
        temp_rr = vitals_data['respiratory_rate']
        temp_bps = vitals_data['blood_pressure_systolic']
        temp_bpm = vitals_data['blood_pressure_mean']
        temp_bpd = vitals_data['blood_pressure_diastolic']
        vitals_SBS = vitals_data['sbs'].flatten()
        hr = []
        rr = []
        SpO2 = []
        bpm = []
        bps = []
        bpd = []
        vitals_list = [hr,rr,SpO2,bpm,bps,bpd]
        vitals_names = ['hr','rr','spo2','bpm','bps','bpd']
        temp_vitals = [temp_hr,temp_rr, temp_SpO2,temp_bpm,temp_bps,temp_bpd] 
        
        flag_list = [0] * (int)(sr * 60 * slice_size_min) #generate a list to insert in place of invalid data, 
        #this list serves as a flag for a window to ignore in the box plot function

        for j in range(len(vitals_list)): #go through every vitals metric
            print(f'original {vitals_names[j]} vitals array shape: {np.array(temp_vitals[j]).shape} ')
            for i in range(len(vitals_SBS)): #go through every SBS score for each vitals metric
                if (Filtering.checkVitals(temp_vitals[j][i], slice_size_min, vitals_names[j])): #check the data in a single window
                    vitals_list[j].append(temp_vitals[j][i]) #append that single window data to the 2D hr,rr,spo2,bpm,bps,bpd arrays if that window's data is valid
                else:
                    vitals_list[j].append(flag_list) #append an array of zeros for window number i for the jth vitals metric if the data is invalid(i.e. too many NaN points)
                    print(f'{vitals_names[j]} SBS index {i} has insufficient data, zeros appended in place') 
            print(f'final {vitals_names[j]} vitals array shape: {np.array(vitals_list[j]).shape}') #should be the number of SBS scores by the number of samples in a window

        #Call Functions for Analysis for each patient here!
        group_neg3 = []
        group_neg2 = []
        group_neg1 = []
        group_0 = []
        group_1 = []
        group_2 = []

        myFunction = lambda signal: np.mean(signal) #can always change


        for k in  range(len(vitals_list)): 
            signal = (np.array(vitals_list[k])) 
            if(np.mean(signal)!=0):
                if(vitals_SBS[k] == -3):
                    group_neg3.append(myFunction(signal))
                elif(vitals_SBS[k] == -2):
                    group_neg2.append(myFunction(signal))
                elif(vitals_SBS[k] == -1):                   
                    group_neg1.append(myFunction(signal))
                elif(vitals_SBS[k] == 0):
                    group_0.append(myFunction(signal))
                elif(vitals_SBS[k] == 1):
                    group_1.append(myFunction(signal))
                else:
                    group_2.append(myFunction(signal))
        # Perform Mann-Whitney U test
        print('Below are Mann-Whitney U test Results. Small P values mean data is dissimilar')
        if(len(group_neg2) > 3 and len(group_1) > 3):
            u_stat, p_value = mannwhitneyu(group_neg2, group_1)
            print(f"U-statistic on SBS -2 vs SBS 1 Means: {u_stat}, P-value: {p_value}")
        if(len(group_neg1) > 3 and len(group_0) > 3):
            u_stat, p_value = mannwhitneyu(group_neg1, group_0)
            print(f"U-statistic on SBS -1 vs SBS 0 Means: {u_stat}, P-value: {p_value}")
        if(len(group_0) > 3 and len(group_1) > 3):
            u_stat, p_value = mannwhitneyu(group_0, group_1)
            print(f"U-statistic on SBS 0 vs SBS 1 Means: {u_stat}, P-value: {p_value}")
